# Thiele Machine Verification Analysis

This notebook loads experiment CSVs, computes scaling fits, and plots the results. It expects a `lab_notebook.csv` file in `experiments/`. Run the analysis after running the baseline experiments.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

LAB = Path('experiments/lab_notebook_template.csv')
df = pd.read_csv(LAB)
df.head()

In [ ]:
# Fit model: time = a * N^b or time = a * exp(b*N)
from scipy.optimize import curve_fit

def poly_model(N, a, b):
    return a * (N ** b)

def exp_model(N, a, b):
    return a * np.exp(b * N)

def fit_and_plot(group):
    N = group['instance_size'].values.astype(float)
    T = group['runtime_seconds'].values.astype(float)
    # Try both fits
    try:
        popt_p, _ = curve_fit(poly_model, N, T, maxfev=10000)
        p_pred = poly_model(N, *popt_p)
    except Exception as e:
        popt_p = None
        p_pred = None
    try:
        popt_e, _ = curve_fit(exp_model, N, T, maxfev=10000)
        e_pred = exp_model(N, *popt_e)
    except Exception as e:
        popt_e = None
        e_pred = None
    plt.figure()
    plt.scatter(N, T, label='data')
    if p_pred is not None:
        plt.plot(N, p_pred, label=f'poly fit a={popt_p[0]:.3g}, b={popt_p[1]:.3g}')
    if e_pred is not None:
        plt.plot(N, e_pred, label=f'exp fit a={popt_e[0]:.3g}, b={popt_e[1]:.3g}')
    plt.legend()
    plt.xlabel('N')
    plt.ylabel('time (s)')
    plt.title(f
    plt.show()
    return popt_p, popt_e

# Example: group by program_type
for name, group in df.groupby('program_type'):
    print('Group:', name)
    fit_and_plot(group)

## Extended Statistical Analysis

This notebook augments the basic scaling plots with model selection (AIC/BIC),
likelihood-ratio style comparisons (via RSS-based approximations), bootstrap
confidence intervals for fitted exponents, residual diagnostics, and
confidence bands for model predictions. Run the notebook after populating the
`experiments/lab_notebook_template.csv` with actual experiment runs.

In [ ]:
# Additional analysis: AIC, BIC, bootstrap, residuals, confidence bands
import numpy as np
from scipy.optimize import curve_fit


def aic_bic(n, rss, k):
    # Gaussian errors approximation: AIC = 2k + n ln(RSS/n)
    aic = 2 * k + n * np.log(rss / n)
    bic = k * np.log(n) + n * np.log(rss / n)
    return aic, bic


def bootstrap_params(model, N, T, n_boot=200, model_name='poly'):
    params = []
    rng = np.random.default_rng(0)
    for _ in range(n_boot):
        idx = rng.choice(len(N), size=len(N), replace=True)
        try:
            if model_name == 'poly':
                popt, _ = curve_fit(model, N[idx], T[idx], maxfev=20000)
            else:
                popt, _ = curve_fit(model, N[idx], T[idx], maxfev=20000)
            params.append(popt)
        except Exception:
            continue
    return np.array(params)


def residuals_and_plots(N, T, popt_p, popt_e, name='group'):
    # Compute residuals and produce diagnostic plots
    import matplotlib.pyplot as plt
    Nf = np.linspace(min(N), max(N), 200)
    p_pred = poly_model(Nf, *popt_p) if popt_p is not None else None
    e_pred = exp_model(Nf, *popt_e) if popt_e is not None else None

    # Residuals for poly
    if popt_p is not None:
        fitted = poly_model(N, *popt_p)
        res = T - fitted
        plt.figure()
        plt.scatter(fitted, res)
        plt.axhline(0, color='k', linestyle='--')
        plt.xlabel('Fitted (poly)'); plt.ylabel('Residual'); plt.title(f'Residuals (poly) - {name}')
        plt.savefig(f'experiments/residuals_poly_{name}.png')

    if popt_e is not None:
        fitted = exp_model(N, *popt_e)
        res = T - fitted
        plt.figure()
        plt.scatter(fitted, res)
        plt.axhline(0, color='k', linestyle='--')
        plt.xlabel('Fitted (exp)'); plt.ylabel('Residual'); plt.title(f'Residuals (exp) - {name}')
        plt.savefig(f'experiments/residuals_exp_{name}.png')

    # Return RMS
    return np.sqrt(np.mean((T - (poly_model(N, *popt_p) if popt_p is not None else 0))**2))

# Integrate with earlier per-group loop
for name, group in df.groupby('program_type'):
    N = group['instance_size'].values.astype(float)
    T = group['runtime_seconds'].values.astype(float)
    if len(N) < 3:
        print('Not enough points for robust fitting for', name)
        continue
    # Fit models
    popt_p, popt_e = None, None
    try:
        popt_p, _ = curve_fit(poly_model, N, T, maxfev=20000)
    except Exception:
        pass
    try:
        popt_e, _ = curve_fit(exp_model, N, T, maxfev=20000)
    except Exception:
        pass

    # RSS and AIC/BIC
    if popt_p is not None:
        rss_p = np.sum((T - poly_model(N, *popt_p))**2)
        aic_p, bic_p = aic_bic(len(N), rss_p, k=2)
    else:
        rss_p, aic_p, bic_p = np.inf, None, None
    if popt_e is not None:
        rss_e = np.sum((T - exp_model(N, *popt_e))**2)
        aic_e, bic_e = aic_bic(len(N), rss_e, k=2)
    else:
        rss_e, aic_e, bic_e = np.inf, None, None

    print(f"Group: {name} | AIC poly: {aic_p} | AIC exp: {aic_e} | BIC poly: {bic_p} | BIC exp: {bic_e}")

    # Bootstrap exponent CI for poly fit
    if popt_p is not None:
        params = bootstrap_params(poly_model, N, T, n_boot=300, model_name='poly')
        if params.size:
            b_vals = params[:, 1]
            low, high = np.percentile(b_vals, [2.5, 97.5])
            print(f'Bootstrap 95% CI for poly exponent b: [{low:.3g}, {high:.3g}]')

    # Residual diagnostics
    residuals_and_plots(N, T, popt_p, popt_e, name=name)

print('Extended analysis complete')